<a href="https://colab.research.google.com/github/Jeong-Yunmi/pytorch-tutorial/blob/main/Lap_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
def train(model, optimizer, criterion):
    total_batch = len(data_loader)
    for epoch in range(training_epochs):
        avg_cost = 0

        for X, Y in data_loader:
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

    print('Learning finished')



def test(model, test_data):
    with torch.no_grad():
        X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
        Y_test = mnist_test.test_labels.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

        r = random.randint(0, len(mnist_test) - 1)
        X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
        Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

        print('Label: ', Y_single_data.item())
        single_prediction = linear(X_single_data)
        print('Prediction: ', torch.argmax(single_prediction, 1).item())

In [8]:
linear = torch.nn.Linear(784, 10, bias=True).to(device)

torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.1953, -0.4404,  0.0139,  ...,  0.7510,  1.5190,  0.9637],
        [ 0.4146, -1.0660, -0.9969,  ..., -0.3259, -0.2695, -1.3316],
        [ 0.2600, -1.1821,  0.0065,  ...,  1.0648, -1.2251,  0.9841],
        ...,
        [ 0.0279, -1.7242, -1.1723,  ...,  1.3651, -0.0689, -0.7931],
        [ 0.0952,  0.1939,  0.1636,  ..., -1.0489,  0.6224, -1.7181],
        [ 2.6504,  0.6297, -1.6571,  ..., -0.0781, -1.1398, -0.8711]],
       requires_grad=True)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [11]:
train(linear, optimizer, criterion)

Epoch: 0001 cost = 2.014962912
Epoch: 0002 cost = 1.153176665
Epoch: 0003 cost = 0.883272231
Epoch: 0004 cost = 0.743711412
Epoch: 0005 cost = 0.656470060
Epoch: 0006 cost = 0.594376326
Epoch: 0007 cost = 0.548456013
Epoch: 0008 cost = 0.512833595
Epoch: 0009 cost = 0.484449595
Epoch: 0010 cost = 0.460656583
Epoch: 0011 cost = 0.441532224
Epoch: 0012 cost = 0.425174266
Epoch: 0013 cost = 0.411161363
Epoch: 0014 cost = 0.398808271
Epoch: 0015 cost = 0.387932807
Learning finished


In [12]:
test(linear, mnist_test)

Accuracy: 0.8924999833106995
Label:  8
Prediction:  3


In [13]:
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [14]:
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 0.1890,  3.3274, -0.6512,  ..., -1.6801, -2.0141, -1.8596],
        [-2.1198, -0.1507,  0.3873,  ..., -0.5323,  0.0938, -0.0894],
        [-1.1228, -1.5286,  0.3924,  ..., -2.4520,  1.8243, -0.1563],
        ...,
        [ 0.0896, -0.0116,  0.4110,  ...,  1.1251, -1.5692,  0.3844],
        [-0.6065, -0.2887,  1.8685,  ..., -1.1130,  0.1281,  0.8384],
        [ 0.7321,  1.2681,  0.3042,  ...,  0.4207,  1.1925,  0.6897]],
       requires_grad=True)

In [15]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
train(model, optimizer, criterion)

Epoch: 0001 cost = 173.244064331
Epoch: 0002 cost = 37.550716400
Epoch: 0003 cost = 23.142995834
Epoch: 0004 cost = 15.941267014
Epoch: 0005 cost = 11.548622131
Epoch: 0006 cost = 8.576098442
Epoch: 0007 cost = 6.423884392
Epoch: 0008 cost = 4.805458069
Epoch: 0009 cost = 3.635167122
Epoch: 0010 cost = 2.697978973
Epoch: 0011 cost = 2.026228189
Epoch: 0012 cost = 1.541051745
Epoch: 0013 cost = 1.113659739
Epoch: 0014 cost = 0.903187156
Epoch: 0015 cost = 0.719041109
Learning finished


In [17]:
test(model, mnist_test)

Accuracy: 0.948199987411499
Label:  5
Prediction:  5


In [18]:
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [20]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.0188, -0.0167,  0.0186,  ...,  0.0885, -0.0417,  0.0386],
        [-0.0333,  0.1441, -0.1184,  ...,  0.0708,  0.1470,  0.0258],
        [ 0.1293, -0.0672, -0.0615,  ..., -0.1252,  0.0461, -0.0106],
        ...,
        [-0.0688, -0.0057,  0.0024,  ..., -0.0749, -0.0592, -0.1087],
        [-0.1047, -0.0179,  0.0540,  ...,  0.0712,  0.0117,  0.0531],
        [ 0.0657,  0.0181,  0.1216,  ..., -0.1369,  0.1237,  0.0790]],
       requires_grad=True)

In [21]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
train(model, optimizer, criterion)

Epoch: 0001 cost = 0.244155169
Epoch: 0002 cost = 0.092878215
Epoch: 0003 cost = 0.059108026
Epoch: 0004 cost = 0.042317916
Epoch: 0005 cost = 0.033395760
Epoch: 0006 cost = 0.026045589
Epoch: 0007 cost = 0.021893093
Epoch: 0008 cost = 0.018082647
Epoch: 0009 cost = 0.014690350
Epoch: 0010 cost = 0.014739043
Epoch: 0011 cost = 0.013285695
Epoch: 0012 cost = 0.014508184
Epoch: 0013 cost = 0.007810565
Epoch: 0014 cost = 0.010716433
Epoch: 0015 cost = 0.011598592
Learning finished


In [23]:
test(model, mnist_test)

Accuracy: 0.9793999791145325
Label:  7
Prediction:  7


In [24]:
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()

In [25]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0970,  0.0634, -0.0758,  ..., -0.0316,  0.0042,  0.0056],
        [ 0.0460,  0.0493,  0.0034,  ...,  0.0885,  0.0308, -0.0336],
        [-0.0832,  0.0494,  0.0022,  ...,  0.1026, -0.0599, -0.0818],
        ...,
        [-0.0687, -0.0283, -0.1044,  ...,  0.0026,  0.0429,  0.0025],
        [ 0.0935, -0.1020, -0.0389,  ...,  0.0847, -0.0401,  0.0996],
        [-0.0074,  0.0085, -0.0481,  ...,  0.0468,  0.0314,  0.0509]],
       requires_grad=True)

In [29]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
train(model, optimizer, criterion)

Epoch: 0001 cost = 0.274519205
Epoch: 0002 cost = 0.088057652
Epoch: 0003 cost = 0.058561075
Epoch: 0004 cost = 0.041251715
Epoch: 0005 cost = 0.030143186
Epoch: 0006 cost = 0.024795476
Epoch: 0007 cost = 0.020499820
Epoch: 0008 cost = 0.018203005
Epoch: 0009 cost = 0.016960109
Epoch: 0010 cost = 0.013996945
Epoch: 0011 cost = 0.013506106
Epoch: 0012 cost = 0.009347801
Epoch: 0013 cost = 0.006566781
Epoch: 0014 cost = 0.013541656
Epoch: 0015 cost = 0.010861987
Learning finished


In [31]:
test(model, mnist_test)

Accuracy: 0.9750999808311462
Label:  9
Prediction:  9
